# Interpersonal Physical Alignment

In [44]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

### Reading the Training files:

In [45]:
#read training files:
df_Spontan1 = pd.read_csv('Nofar636759792183061005Spontan.csv')
df_Spontan2= pd.read_csv('Nofar636759795182793299Spontan.csv')
df_Sync1= pd.read_csv('Nofar636759794200223541Sync.csv')
df_Sync2= pd.read_csv('Nofar636759797397919664Sync.csv')
df_Alone1_left= pd.read_csv('Nofar636759793193137655Alone.csv')
df_Alone2_left= pd.read_csv('Nofar636759796290435160Alone.csv')
df_Alone_right=pd.read_csv('HandRight.csv')

### Auxiliary Functions:

In [46]:
#1.Preparation of the data set:
def Data_set(ds):
    #rename columns
    rename_columns(ds)
    
    #Delete unnecessary columns
    ds=ds.drop(columns={'Time','Frame_ID','hands'})
    
    #Delete the first seven seconds
    right=pd.concat([ds[ds['Hand_Type']=='right'][400:]], ignore_index=True)
    left=pd.concat([ds[ds['Hand_Type']=='left'][400:]],ignore_index=True)
    
    #Delete unnecessary columns
    right=right.drop(columns={'Hand_Type'})
    left=left.drop(columns={'Hand_Type'}) 
    
    right1=pd.concat([right[::5]],ignore_index=True)
    right2=pd.concat([right[1::5]],ignore_index=True)
    right3=pd.concat([right[2::5]],ignore_index=True)


    left1=pd.concat([left[::5]],ignore_index=True)
    left2=pd.concat([left[1::5]],ignore_index=True)
    left3=pd.concat([left[2::5]],ignore_index=True)
    
    #rename columns right:
    rename_columns_1(rename_columns_right(right1))
    rename_columns_2(rename_columns_right(right2))
    rename_columns_3(rename_columns_right(right3))
    
    right_finish=pd.concat([right1,right2,right3], axis=1)
    
    ##rename columns left:
    rename_columns_1(rename_columns_left(left1))
    rename_columns_2(rename_columns_left(left2))
    rename_columns_3(rename_columns_left(left3))
    left_finish=pd.concat([left1,left2,left3], axis=1)
    
    #Right and left connection:
    ds_end=pd.concat([left_finish,right_finish], axis=1)
    
    return ds_end

In [47]:
#1.1 Preparation of the data set:
def Data_set_alone(ds1,ds2):
    #rename columns
    rename_columns(ds1)
    rename_columns(ds2)
    
    #Delete unnecessary columns
    ds1=ds1.loc[400:]
    ds2=ds2.loc[400:]

    #Delete the first seven seconds
    ds1=ds1.drop(columns={'Time','Frame_ID','hands','Hand_Type'})#Alone 1
    ds2= ds2.drop(columns={'Time','Frame_ID','hands','Hand_Type'})

    ds2_right1=pd.concat([ds2[::5]],ignore_index=True)
    ds2_right2=pd.concat([ds2[1::5]],ignore_index=True)
    ds2_right3=pd.concat([ds2[2::5]],ignore_index=True)
    
    ds1_left1=pd.concat([ds1[::5]],ignore_index=True)
    ds1_left2=pd.concat([ds1[1::5]],ignore_index=True)
    ds1_left3=pd.concat([ds1[2::5]],ignore_index=True)
    
    #rename columns right:
    rename_columns_1(rename_columns_right(ds2_right1))
    rename_columns_2(rename_columns_right(ds2_right2))
    rename_columns_3(rename_columns_right(ds2_right3))
    ds2_right=pd.concat([ds2_right1,ds2_right2,ds2_right3], axis=1)
    ds2_right

    #rename columns left:
    rename_columns_1(rename_columns_left(ds1_left1))
    rename_columns_2(rename_columns_left(ds1_left2))
    rename_columns_3(rename_columns_left(ds1_left3))
    ds1_left=pd.concat([ds1_left1,ds1_left2,ds1_left3], axis=1)
    ds1_left

    df=pd.concat([ds1_left,ds2_right], axis=1)
    return df

In [48]:
#2.rename columns:
def rename_columns(ds):
    ds.rename(columns = {' Hand Type': 'Hand_Type', ' Frame ID': 'Frame_ID',' # hands': 'hands',' Position X':'Position_X',
                         ' Position Y':'Position_Y', ' Position Z':'Position_Z', ' Velocity X':'Velocity_X' ,
                         ' Velocity Y':'Velocity_Y',' Velocity Z': 'Velocity_Z', ' Pitch':'Pitch',' Roll':'Roll', 
                         ' Yaw': 'Yaw',' Wrist Pos X': 'Wrist_Pos_X',' Wrist Pos Y': 'Wrist_Pos_Y',
                         ' Wrist Pos Z': 'Wrist_Pos_Z',' Elbow pos X':'Elbow_pos_X',' Elbow Pos Y':'Elbow_Pos_Y',
                         ' Elbow Pos Z':'Elbow_Pos_Z', ' Grab Strenth':'Grab_Strenth', ' Grab Angle':'Grab_Angle',
                         ' Pinch Strength': 'Pinch_Strength' }, inplace = True)

    return ds

In [49]:
#3.rename columns right:
def rename_columns_right(ds):
    ds.columns =['right_' + col.strip() for col in ds.columns]
    return ds

In [50]:
#4.rename columns left:
def rename_columns_left(ds):
    ds.columns =['left_' + col.strip() for col in ds.columns]
    return ds

In [51]:
#5.rename columns '1':
def rename_columns_1(ds):
    ds.columns =['1_' + col.strip() for col in ds.columns]
    return ds

In [52]:
#6.rename columns '2':
def rename_columns_2(ds):
    ds.columns =['2_' + col.strip() for col in ds.columns]
    return ds

In [53]:
#7.rename columns '3':
def rename_columns_3(ds):
    ds.columns =['3_' + col.strip() for col in ds.columns]
    return ds

### Preparing The Training Set:

Marking for each situation:
alone-0
spontan-1
sync-2

In [54]:
df_data1=Data_set(df_Spontan1)
df_data1['Situation']=1
df_data1.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-189.2008,201.5695,118.83140,51.15285,7.851839,135.06990,2.547028,-2.064779,2.266571,-234.8915,...,168.6342,191.6638,53.61215,318.0985,-9.967384,107.0739,0.0,0.000000,0.0,1
1,-188.5781,201.7215,121.62000,17.13529,-21.202150,-69.78037,2.466415,-2.282879,2.093641,-239.5060,...,169.9814,196.3262,57.88356,323.0591,1.559237,125.0136,0.0,0.058250,0.0,1
2,-182.8263,199.6404,111.63400,77.72032,-109.283900,-195.41560,2.061066,-2.711452,1.715273,-245.2970,...,170.2504,201.4257,67.64161,351.8581,29.396430,125.0176,0.0,0.343037,0.0,1
3,-179.5846,196.4469,97.78147,-14.85708,28.873710,-173.71890,1.169149,-3.088204,1.505516,-246.5112,...,173.9505,212.8767,82.48917,351.0408,31.386950,122.0858,0.0,0.237095,0.0,1
4,-173.6773,185.8959,79.04739,-174.80290,-41.206230,-226.52820,1.387681,-2.820010,1.493371,-233.0349,...,178.9406,223.2725,100.26940,361.4464,45.564430,131.5566,0.0,0.094568,0.0,1


In [55]:
df_data2=Data_set(df_Spontan2)
df_data2['Situation']=1
df_data2.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-126.6344,271.4067,-2.982844,-28.35983,2.332575,-263.47380,1.674827,0.688811,1.660310,-180.0166,...,197.5770,207.9238,88.12614,346.8222,8.104187,120.5078,0.328424,1.212852,0.000000,1
1,-133.7904,285.3613,-17.826840,-35.78001,152.157400,-56.10892,1.774164,0.629482,1.728857,-188.5895,...,197.6436,205.2110,84.97773,342.0772,0.713542,108.3927,0.085506,0.949489,0.000000,1
2,-137.2090,293.8431,-13.481750,-48.33509,143.196400,131.77320,1.864380,0.682745,1.847740,-186.4583,...,202.4893,210.9184,90.02805,337.9912,2.475752,127.7035,0.000000,0.683776,0.000000,1
3,-145.2136,307.0414,3.549840,-71.19623,129.084000,250.76500,2.049705,0.571551,2.030593,-191.2853,...,209.6395,219.5450,99.45814,350.6357,14.141310,133.4866,0.000000,0.774415,0.000000,1
4,-151.9046,313.3191,22.051200,-87.34524,45.946650,245.15310,2.261394,0.479691,2.214082,-194.9129,...,212.8856,224.3641,106.82270,336.0346,5.528465,120.0068,0.302414,1.262033,0.117483,1


In [56]:
df_data3=Data_set(df_Sync1)
df_data3['Situation']=2
df_data3.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-206.8408,393.5508,73.91809,-5.644746,189.08290,75.98833,1.800512,0.868751,1.823949,-256.1260,...,136.2645,250.3768,86.97634,306.2585,65.98695,111.78600,0.0,0.148869,0.0,2
1,-208.3587,409.6695,79.45882,-21.825400,172.45250,56.76864,1.809578,0.913237,1.850775,-256.2855,...,138.1835,258.6602,86.39227,307.6181,74.06355,113.39440,0.0,0.143416,0.0,2
2,-214.2718,432.3884,86.66608,-120.450900,316.32700,80.29108,1.808662,0.929456,1.866255,-260.4548,...,140.8212,263.7460,85.54481,309.7949,78.69095,112.29580,0.0,0.045376,0.0,2
3,-221.5797,460.4847,93.66588,35.752320,76.53919,41.80283,1.816409,0.946383,1.881920,-267.5364,...,141.8221,261.2630,84.78104,308.1611,72.88274,103.68400,0.0,0.000000,0.0,2
4,-218.4883,460.5802,95.68539,-1.438428,30.29321,10.31041,1.827057,0.986202,1.924045,-261.9832,...,142.4756,257.5783,82.37131,296.1019,58.52732,99.42355,0.0,0.009514,0.0,2


In [57]:
df_data4=Data_set(df_Sync2)
df_data4['Situation']=2
df_data4.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-107.8942,242.5804,-25.92258,53.74348,-58.30304,-59.29108,1.407424,1.093573,1.281827,-145.2814,...,176.7192,191.4049,42.18935,336.8761,1.041897,84.86592,0.0,0.137091,0.0,2
1,-107.6104,242.9431,-24.68845,8.82978,-15.09236,108.30720,1.396802,1.121792,1.247923,-143.6024,...,178.0241,197.4730,57.63420,345.6027,13.268470,98.81319,0.0,0.112325,0.0,2
2,-110.3755,247.5872,-9.48944,-79.62490,149.81760,250.62730,1.459334,1.183481,1.318601,-142.6663,...,181.1295,203.3371,76.01127,358.6082,29.393210,120.21990,0.0,0.227633,0.0,2
3,-116.7485,265.0172,17.04470,-92.97170,222.67830,340.46400,1.540220,1.144744,1.504555,-150.1004,...,182.4321,209.0393,94.36221,352.5072,25.048850,124.88370,0.0,0.215206,0.0,2
4,-119.9722,273.4659,46.04023,-58.06327,102.65160,357.37370,1.631299,1.113592,1.697036,-153.8712,...,181.3281,212.0186,106.45620,347.0755,22.982350,128.87610,0.0,0.262893,0.0,2


In [58]:
df_data5=Data_set_alone(df_Alone1_left,df_Alone_right)
df_data5['Situation']=0
df_data5.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-47.48721,184.7444,54.79115,-13.41213,33.88198,-4.199358,2.668306,0.138649,1.860213,-114.4063,...,72.09140,223.7464,45.37529,277.6825,134.4232,194.9290,0.0,0.333224,0.0,0
1,-48.83967,187.9410,54.93202,13.96471,24.14419,8.602594,2.409480,0.232778,1.937676,-112.3089,...,76.88019,209.1200,47.37424,284.2625,121.8424,195.5516,0.0,0.217961,0.0,0
2,-47.33452,187.5269,54.54879,60.56086,-42.17284,-19.543800,2.322831,0.254347,1.976496,-107.9547,...,78.96444,205.9445,48.65848,287.9438,122.7002,196.8276,0.0,0.197606,0.0,0
3,-47.79349,188.6025,52.97571,-18.94626,60.23830,-8.395280,2.334469,0.224934,1.950392,-109.3385,...,77.48059,206.5085,48.14270,286.1384,127.6335,199.0540,0.0,0.272727,0.0,0
4,-51.81873,190.8808,50.54248,-46.90074,-44.48662,-29.608270,2.247233,0.341400,1.930716,-113.0003,...,74.46141,202.4562,44.28679,282.6278,140.9712,203.6773,0.0,0.626321,0.0,0


In [59]:
df_data6=Data_set_alone(df_Alone2_left,df_Alone_right)
df_data6['Situation']=0
df_data6.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-34.91571,290.8873,45.44592,-21.940480,399.14940,107.83190,1.834515,0.606086,1.753849,-95.29629,...,72.09140,223.7464,45.37529,277.6825,134.4232,194.9290,0.0,0.333224,0.0,0
1,-39.43504,311.7145,51.18405,-41.696370,246.87730,95.23635,1.835050,0.592120,1.742382,-100.97960,...,76.88019,209.1200,47.37424,284.2625,121.8424,195.5516,0.0,0.217961,0.0,0
2,-41.89442,322.7849,58.73740,-1.047656,-70.33784,99.00803,1.893666,0.585736,1.776199,-103.08620,...,78.96444,205.9445,48.65848,287.9438,122.7002,196.8276,0.0,0.197606,0.0,0
3,-40.76907,311.1992,59.16417,56.171670,-224.79290,-55.20494,1.916365,0.593767,1.797421,-101.34170,...,77.48059,206.5085,48.14270,286.1384,127.6335,199.0540,0.0,0.272727,0.0,0
4,-30.96019,278.9513,50.96684,137.107200,-453.02630,-115.94690,1.892802,0.557393,1.766450,-93.14972,...,74.46141,202.4562,44.28679,282.6278,140.9712,203.6773,0.0,0.626321,0.0,0


### Connecting The Entire Training Set:


In [60]:
df_train=pd.concat([df_data1,df_data2,df_data3,df_data4,df_data5,df_data6],axis=0)
df_train.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-189.2008,201.5695,118.83140,51.15285,7.851839,135.06990,2.547028,-2.064779,2.266571,-234.8915,...,168.6342,191.6638,53.61215,318.0985,-9.967384,107.0739,0.0,0.000000,0.0,1
1,-188.5781,201.7215,121.62000,17.13529,-21.202150,-69.78037,2.466415,-2.282879,2.093641,-239.5060,...,169.9814,196.3262,57.88356,323.0591,1.559237,125.0136,0.0,0.058250,0.0,1
2,-182.8263,199.6404,111.63400,77.72032,-109.283900,-195.41560,2.061066,-2.711452,1.715273,-245.2970,...,170.2504,201.4257,67.64161,351.8581,29.396430,125.0176,0.0,0.343037,0.0,1
3,-179.5846,196.4469,97.78147,-14.85708,28.873710,-173.71890,1.169149,-3.088204,1.505516,-246.5112,...,173.9505,212.8767,82.48917,351.0408,31.386950,122.0858,0.0,0.237095,0.0,1
4,-173.6773,185.8959,79.04739,-174.80290,-41.206230,-226.52820,1.387681,-2.820010,1.493371,-233.0349,...,178.9406,223.2725,100.26940,361.4464,45.564430,131.5566,0.0,0.094568,0.0,1


Clearing the data set:

In [61]:
#NaN values
df_train.isnull().sum()

1_left_Position_X         112
1_left_Position_Y         112
1_left_Position_Z         112
1_left_Velocity_X         112
1_left_Velocity_Y         112
                         ... 
3_right_Elbow_Pos_Z       169
3_right_Grab_Strenth       17
3_right_Grab_Angle         17
3_right_Pinch_Strength     17
Situation                   0
Length: 109, dtype: int64

In [62]:
#Deleting NaN values:
df_train=df_train.dropna()

In [63]:
#The size of the training set:
df_train.shape

(4901, 109)

split the dataset into X_train and y_train:

In [64]:
X_train = df_train[df_train.columns[:-1]]
y_train = df_train[df_train.columns[-1]].values

Data normalization:

In [65]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.values)

### Reading the Testing files:

In [66]:
#read test files:

TSpontan1=pd.read_csv('Oded636754667262994505Spontan.csv')
TSpontan2=pd.read_csv('Oded636754670490266956Spontan.csv')
TAlone1_left=pd.read_csv('Oded636754665152715825Alone.csv')
TAlone2_left=pd.read_csv('Oded636754669540564461Alone.csv')
TAlone_right=pd.read_csv('HandRight.csv')
TSync1=pd.read_csv('Oded636754668465917901Sync.csv')
TSync2=pd.read_csv('Oded636754671560850581Sync.csv')

### Preparing The Testing Set:

In [67]:
df_test1=Data_set(TSpontan1)
df_test1['Situation']=1
df_test1.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-159.4710,207.8980,37.86498,12.614280,21.040960,11.664840,0.883790,0.781852,0.292336,-184.7050,...,234.3865,205.6096,81.26472,349.0020,-30.04540,70.09523,0.000000,0.835781,0.118366,1
1,-162.2006,208.5295,37.47084,-2.995856,-8.655634,-7.248703,0.857456,0.522295,0.356639,-189.2219,...,250.8236,214.5506,86.70105,372.8465,-17.15554,101.40450,0.000000,0.566919,0.000000,1
2,-160.2693,206.0383,35.84936,9.736940,-24.615120,-17.633990,0.848466,0.597228,0.311476,-185.8938,...,240.2099,203.1088,68.53699,358.3589,-30.27212,87.50537,0.000000,0.331445,0.000000,1
3,-155.4185,197.8061,27.32217,60.717900,-46.558760,-95.912350,0.653895,1.000264,0.403417,-189.5818,...,245.0147,201.0733,37.08168,323.8184,-46.83382,69.69169,0.178223,1.261577,0.000000,1
4,-160.3516,208.5144,16.01331,-42.826350,124.581900,-137.578600,0.560986,1.794768,0.449844,-189.9952,...,243.3038,193.8497,23.68001,315.4832,-53.12626,73.92340,0.000000,0.757096,0.010522,1


In [68]:
df_test2=Data_set(TSpontan2)
df_test2['Situation']=1
df_test2.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-122.42950,214.1841,-2.355883,-45.98295,38.56421,37.515060,0.756515,1.114662,0.302690,-148.7184,...,248.3764,196.1565,41.22389,437.1819,17.47952,55.914890,0.00000,1.048550,0.0,1
1,-117.42330,245.9741,-2.076282,62.05891,177.45000,-3.807844,0.647368,0.509733,0.654221,-163.0690,...,256.0562,212.5190,39.61054,422.5826,12.80279,53.253000,0.00426,0.975459,0.0,1
2,-106.74170,278.8350,7.754387,131.13840,171.43220,49.734500,0.215910,0.087277,1.007976,-170.8958,...,236.2335,203.9041,18.35608,377.4212,-14.57490,5.325418,0.00000,0.497224,0.0,1
3,-97.28513,304.1839,19.334730,29.07671,229.75710,54.855430,-0.170631,-0.158785,1.134916,-164.4684,...,214.5435,183.3091,12.61273,380.4264,-17.12918,24.447500,0.00000,0.436049,0.0,1
4,-96.55764,319.9968,22.978480,-11.77001,88.04864,-12.829320,-0.035772,-0.154000,1.043252,-162.6717,...,213.5376,184.1011,24.96937,382.7198,-11.99137,52.054600,0.00000,0.455628,0.0,1


In [69]:
df_test3=Data_set(TSync1)
df_test3['Situation']=2
df_test3.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-128.7858,257.7906,52.98791,-25.73314,220.62970,-30.248790,0.742216,1.376752,0.613964,-167.2924,...,213.3031,184.0184,75.53546,343.4903,-42.385110,103.8306,0.0,0.094790,0.0,2
1,-138.3775,272.0782,66.09569,29.56347,45.26238,42.362370,1.383295,0.960545,1.240112,-181.1171,...,212.2393,185.4836,79.20396,348.0452,-35.470390,120.9648,0.0,0.044036,0.0,2
2,-133.4747,259.9460,70.22956,15.50605,-33.15944,5.354039,1.617877,0.963283,1.632516,-182.7689,...,219.2139,205.0249,80.89642,331.1379,-30.795980,110.3817,0.0,0.113714,0.0,2
3,-128.9518,248.7449,53.06909,98.12008,-143.22840,-115.018700,1.075543,1.058067,0.806269,-169.0929,...,218.7967,220.7636,84.51604,348.5232,-7.511449,92.8456,0.0,0.091865,0.0,2
4,-120.5431,238.9206,42.26253,-12.11274,23.59064,-19.062790,0.524013,1.311944,0.616640,-163.3460,...,216.8362,236.5278,92.31183,356.0240,14.285260,76.7275,0.0,0.087023,0.0,2


In [70]:
df_test4=Data_set(TSync2)
df_test4['Situation']=2
df_test4.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-107.1554,205.4341,52.63971,-33.068810,-54.64933,19.564360,1.251578,0.908294,1.192183,-157.4995,...,212.0952,158.4471,64.58427,417.5757,30.517520,164.13170,0.0,0.527675,0.0,2
1,-105.6995,185.3914,54.46416,12.538710,-131.21670,28.427160,1.081261,0.942450,1.000975,-155.3438,...,208.6106,157.2963,61.03529,410.8712,12.924940,143.16610,0.0,0.693424,0.0,2
2,-106.7319,169.7657,51.85217,-34.048190,26.67254,-24.756730,1.041653,0.919439,0.975252,-156.0760,...,207.3662,180.0729,60.97885,404.4130,8.840608,79.24637,0.0,0.503026,0.0,2
3,-108.8834,179.2030,50.51744,0.316390,107.95330,-3.101729,1.044647,0.953506,0.982089,-157.2386,...,205.8788,195.3419,61.51339,383.5623,3.566986,51.12390,0.0,0.178204,0.0,2
4,-111.1314,209.2670,52.07385,3.761246,195.84990,-7.340401,1.124856,0.969373,1.047345,-158.5836,...,203.3426,197.7323,63.77743,391.8548,16.758230,76.76105,0.0,0.379660,0.0,2


In [71]:
df_test5=Data_set_alone(TAlone1_left,TAlone_right)
df_test5['Situation']=0
df_test5.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-48.53000,250.9377,24.93804,-7.200670,-60.498630,-12.571580,0.045322,0.018117,0.442760,-88.42228,...,72.09140,223.7464,45.37529,277.6825,134.4232,194.9290,0.0,0.333224,0.0,0
1,-49.28000,248.2912,22.67589,0.014007,-6.823483,-11.143310,0.044318,0.021544,0.445756,-89.12061,...,76.88019,209.1200,47.37424,284.2625,121.8424,195.5516,0.0,0.217961,0.0,0
2,-46.12857,245.7282,22.14544,26.048630,-40.233140,-34.158400,0.074632,0.006538,0.500195,-88.46332,...,78.96444,205.9445,48.65848,287.9438,122.7002,196.8276,0.0,0.197606,0.0,0
3,-46.35461,248.6904,20.29183,0.182497,7.313391,1.241604,0.109030,-0.062844,0.511641,-88.84023,...,77.48059,206.5085,48.14270,286.1384,127.6335,199.0540,0.0,0.272727,0.0,0
4,-45.94225,248.2820,20.15773,7.453786,-2.372849,-3.269597,0.102823,-0.047912,0.509757,-88.49717,...,74.46141,202.4562,44.28679,282.6278,140.9712,203.6773,0.0,0.626321,0.0,0


In [72]:
df_test6=Data_set_alone(TAlone2_left,TAlone_right)
df_test6['Situation']=0
df_test6.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-71.26089,261.9890,30.63798,-85.28862,-29.82305,8.991480,0.247211,0.746355,0.997454,-135.71250,...,72.09140,223.7464,45.37529,277.6825,134.4232,194.9290,0.0,0.333224,0.0,0
1,-72.04700,240.9700,34.67071,28.84589,-89.65389,54.301350,-0.017571,1.346992,1.028402,-137.63110,...,76.88019,209.1200,47.37424,284.2625,121.8424,195.5516,0.0,0.217961,0.0,0
2,-53.95756,228.1918,36.93213,123.16810,57.32764,7.092518,0.044919,1.844116,1.035042,-118.90720,...,78.96444,205.9445,48.65848,287.9438,122.7002,196.8276,0.0,0.197606,0.0,0
3,-38.41958,237.2370,40.69280,71.32862,54.56944,44.294750,0.243513,2.206802,0.938364,-97.63422,...,77.48059,206.5085,48.14270,286.1384,127.6335,199.0540,0.0,0.272727,0.0,0
4,-31.44176,240.1179,40.25331,-37.15764,86.23321,-13.269740,0.283991,2.436616,0.707842,-78.05408,...,74.46141,202.4562,44.28679,282.6278,140.9712,203.6773,0.0,0.626321,0.0,0


### Connecting The Entire Testing Set:

In [73]:
df_test=pd.concat([df_test1,df_test2,df_data3,df_test4,df_test5,df_test6],axis=0)
df_test.head()

,1_left_Position_X,1_left_Position_Y,1_left_Position_Z,1_left_Velocity_X,1_left_Velocity_Y,1_left_Velocity_Z,1_left_Pitch,1_left_Roll,1_left_Yaw,1_left_Wrist_Pos_X,...,3_right_Wrist_Pos_X,3_right_Wrist_Pos_Y,3_right_Wrist_Pos_Z,3_right_Elbow_pos_X,3_right_Elbow_Pos_Y,3_right_Elbow_Pos_Z,3_right_Grab_Strenth,3_right_Grab_Angle,3_right_Pinch_Strength,Situation
0,-159.4710,207.8980,37.86498,12.614280,21.040960,11.664840,0.883790,0.781852,0.292336,-184.7050,...,234.3865,205.6096,81.26472,349.0020,-30.04540,70.09523,0.000000,0.835781,0.118366,1
1,-162.2006,208.5295,37.47084,-2.995856,-8.655634,-7.248703,0.857456,0.522295,0.356639,-189.2219,...,250.8236,214.5506,86.70105,372.8465,-17.15554,101.40450,0.000000,0.566919,0.000000,1
2,-160.2693,206.0383,35.84936,9.736940,-24.615120,-17.633990,0.848466,0.597228,0.311476,-185.8938,...,240.2099,203.1088,68.53699,358.3589,-30.27212,87.50537,0.000000,0.331445,0.000000,1
3,-155.4185,197.8061,27.32217,60.717900,-46.558760,-95.912350,0.653895,1.000264,0.403417,-189.5818,...,245.0147,201.0733,37.08168,323.8184,-46.83382,69.69169,0.178223,1.261577,0.000000,1
4,-160.3516,208.5144,16.01331,-42.826350,124.581900,-137.578600,0.560986,1.794768,0.449844,-189.9952,...,243.3038,193.8497,23.68001,315.4832,-53.12626,73.92340,0.000000,0.757096,0.010522,1


In [74]:
#NaN values
df_test.isnull().sum()

1_left_Position_X         1037
1_left_Position_Y         1037
1_left_Position_Z         1037
1_left_Velocity_X         1037
1_left_Velocity_Y         1037
                          ... 
3_right_Elbow_Pos_Z        158
3_right_Grab_Strenth         6
3_right_Grab_Angle           6
3_right_Pinch_Strength       6
Situation                    0
Length: 109, dtype: int64

In [75]:
#Deleting NaN values:
df_test=df_test.dropna()

In [76]:
#The size of the testing set:
df_test.shape

(2662, 109)

split the dataset into X_test and y_test:

In [77]:
X_test = df_test[df_test.columns[:-1]]
y_test = df_test[df_test.columns[-1]].values

Data normalization:

In [78]:
X_test=scaler.transform(X_test)

### Models

### 1.Adaboost

1.1. Cross Validation:

In [79]:
#1.Adaboost-1
clf_1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=50,algorithm="SAMME.R", learning_rate=0.5)
scores1 = cross_val_score(clf_1, X_train, y_train, cv=3, scoring="accuracy")
print(scores1)
print(scores1.mean())

[0.71804281 0.80159216 0.72749541]
0.7490434607824227


In [80]:
#2.Adaboost-2
clf_2 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4), n_estimators=300,algorithm="SAMME.R", learning_rate=0.5)
scores2 = cross_val_score(clf_2, X_train, y_train, cv=3, scoring="accuracy")
print(scores2)
print(scores2.mean())

[0.85382263 0.90263319 0.83894672]
0.8651341814125456


1.2.Fitting the best model:

In [81]:
clf_2.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=4,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

1.3. Testing the best model

In [82]:
y_pred_ada=clf_2.predict(X_test)
print(confusion_matrix(y_test, y_pred_ada))
print(classification_report(y_test,y_pred_ada))

[[737   0   3]
 [  0 501 236]
 [  0 187 998]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       740
           1       0.73      0.68      0.70       737
           2       0.81      0.84      0.82      1185

    accuracy                           0.84      2662
   macro avg       0.84      0.84      0.84      2662
weighted avg       0.84      0.84      0.84      2662



### 2. Random Forest

2.1. Cross Validation:

In [83]:
RF_1 = RandomForestClassifier(n_estimators=100)
scores4 = cross_val_score(RF_1, X_train, y_train, cv=3, scoring="accuracy")
print(scores4)
print(scores4.mean())

[0.8733945  0.9308022  0.82180037]
0.875332355788768


In [84]:
RF_2 = RandomForestClassifier(n_estimators=500)
scores5= cross_val_score(RF_2, X_train, y_train, cv=3, scoring="accuracy")
print(scores5)
print(scores5.mean())

[0.87767584 0.93263931 0.82363748]
0.8779842107201558


2.2.Fitting the best model:

In [85]:
RF_2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

2.3. Testing the best model:

In [86]:
y_pred_RF=RF_2.predict(X_test)
print(confusion_matrix(y_test, y_pred_RF))
print(classification_report(y_test,y_pred_RF))

[[740   0   0]
 [  0 638  99]
 [  0 226 959]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       740
           1       0.74      0.87      0.80       737
           2       0.91      0.81      0.86      1185

    accuracy                           0.88      2662
   macro avg       0.88      0.89      0.88      2662
weighted avg       0.89      0.88      0.88      2662

